# API Gateway Demo

In [ ]:
import os
from dotenv import load_dotenv

from data_prep import (
    populate_bucket,
    create_dummy_job_objects,
)
from client_utils import (
    get_job_progress,
    get_job_results,
    get_overall_progress,
    update_job_results,
    submit_job,
)

load_dotenv()
api_key = os.environ["API_KEY"]
api_url = "https://yrzic1qgg2.execute-api.us-east-1.amazonaws.com/dev/api"

## Populate bucket

In [ ]:
uploads_bucket_name = "ai-description-dev-nt-isngd-2-381491992967-uploads"
image_s3_uri, original_metadata_s3_uri, context_s3_uri = populate_bucket(
    bucket_name=uploads_bucket_name,
    image_fpath="../tests/data/offensive_image.jpg",
)

## Create bias job

In [ ]:
job_name = "job_001"
bias_job_name = f"{job_name}_bias"
metadata_job_name = f"{job_name}_metadata"

bias_job_objects = create_dummy_job_objects(
    job_name=bias_job_name,
    original_metadata_s3_uri=original_metadata_s3_uri,
    context_s3_uri=context_s3_uri,
    image_s3_uri=image_s3_uri,
)
submit_job(
    api_url=api_url,
    job_name=job_name,
    job_type="bias",
    works=bias_job_objects,
    api_key=api_key,
)

## Create metadata job

In [ ]:
metadata_job_objects = create_dummy_job_objects(
    job_name=metadata_job_name,
    original_metadata_s3_uri=original_metadata_s3_uri,
    context_s3_uri=context_s3_uri,
    image_s3_uri=image_s3_uri,
)
submit_job(
    api_url=api_url,
    job_name=job_name,
    job_type="metadata",
    works=bias_job_objects,
    api_key=api_key,
)

## Check Bias Job Progress

In [ ]:
bias_progress = get_job_progress(
    api_url=api_url,
    job_name=bias_job_name,
    session_token=get_session_token(**session_token_kwargs),
)
for key, val in bias_progress.items():
    print(f"{key}: {val}")

## Check metadata job progress

In [ ]:
metadata_progress = get_job_progress(
    api_url=api_url,
    job_name=metadata_job_name,
    api_key=api_key,
)
for key, val in metadata_progress.items():
    print(f"{key}: {val}")

## Get overall progress

In [ ]:
overall_progress = get_overall_progress(
    api_url=api_url, api_key=api_key
)
for key, val in overall_progress.items():
    print(f"{key}: {val}")

## Get Bias Job Results

In [ ]:
item = get_job_results(
    api_url=api_url,
    job_name=bias_job_name,
    work_id=bias_progress["job_progress"]["READY FOR REVIEW"][0],
    api_key=api_key,
)
for key, val in item.items():
    print(f"{key}: {val}")
    print()

item = get_job_results(
    api_url=api_url,
    job_name=bias_job_name,
    work_id=bias_progress["job_progress"]["READY FOR REVIEW"][1],
    api_key=api_key,
)
for key, val in item.items():
    print(f"{key}: {val}")
    print()

## Get metadata job results

In [ ]:
item = get_job_results(
    api_url=api_url,
    job_name=metadata_job_name,
    work_id=metadata_progress["job_progress"]["READY FOR REVIEW"][0],
    api_key=api_key,
)
for key, val in item.items():
    print(f"{key}: {val}")
    print()

## Update Job Results

In [ ]:
update_job_results(
    api_url=api_url,
    job_name=job_name,
    work_id=bias_progress["READY_FOR_REVIEW"][0],
    api_key=api_key,
)